# Реализация словаря на Python

Здесь я покажу,как создать словарь(dict),используя список.

Словарь в Python является ассоциативным массивом, то есть хранит данные в виде пар (ключ, значение). Словарь – измененяемый тип данных. Это значит, что в него можно добавлять элементы, изменять их и удалять из словаря. Он также предоставляет операцию поиска и возвращения элемента по ключу.
Как известно, реализация хэш-таблицы должна учитывать возможность появления коллизий – ситуаций, когда разные ключи имеют одинаковое значение хэша. Должен быть способ вставки и извлечения элементов с учётом коллизий.
В моем примере hashmap является списком списков(бакетов),которые в свою очередь,являются списком кортежей(слотов),содержащие в себе пары ключ-значение.
Пример структуры:

In [1]:
[
    [("key1","value")],
    [("key2","value"),("key5","value")],
    [("key3","value")],
    [("key4","value")]
]

[[('key1', 'value')],
 [('key2', 'value'), ('key5', 'value')],
 [('key3', 'value')],
 [('key4', 'value')]]

Смысл в том,что каждый слот,внутри бакета содержит (key, value)кортеж или "пару".
Теперь,когда нам понятно,как мы будем структурировать данные,нам просто необходимо понять алгоритм работы с каждой операцией.В общем понимании,это :


Преобразование ключа в целое число с помощью функции хэширования: hash_key.

Преобразовать этот хэш в индекс бакета с помощью операции - остаток от деления.

Получить этот бакет из списка aMap, а затем найти именно тот слот, который содержит искомый ключ.

В случае set мы делаем это, чтобы заменить дубликат ключей,или для добавления новых.


# new
Сначала создаю функцию-конструктор. Я создала переменную aMap - пустой список, затем, положила num_buckets списков внутрь него.
Эти бакеты будут использоваться для хранения содержимого hashmap, как набор(key, value). 

In [ ]:
def new(num_buckets=256):
    aMap = []
    for i in range(0, num_buckets):
        aMap.append([])
    return aMap

# hash_key
Это обманчиво простая функция- ядро, показывающее, как словарь работает. Она использует, встроенную в Python хэш-функцию для преобразования строки в число.
Я использую остаток от деления хэша ключа на длину aMap, чтобы получить бакет с этим ключем.

In [2]:
def hash_key(aMap, key):
    return hash(key) % len(aMap)

# get_bucket
Эта функция использует hash_key, чтобы найти бакет, в котором может быть ключ.Так как я использовала  % len(aMap), я знаю, что bucket_id который я получаю впишется в дипазон индексов aMap.Потом, используя bucket_id я смогу получить бакет, где хранится ключ.

In [ ]:
def get_bucket(aMap, key):
    bucket_id = hash_key(aMap, key)
    return aMap[bucket_id]

# get_slot
Эта функция использует get_bucket чтобы получить бакет, и затем она проходится через каждый его элемент, пока не найдет соответствующий ключ.Как только это происходит,она возвращает кортеж (i, k, v) в котором хранится индекс, ключ, значение.


In [ ]:
def get_slot(aMap, key, default=None):
    bucket = get_bucket(aMap, key)

    for i, kv in enumerate(bucket):
        k, v = kv
        if key == k:
            return i, k, v

    return -1, key, default

# get
Это "удобная" функциия, которая делает то, что большинство людей хотят от HashMap. Она использует get_slot чтобы получить (i, к, v), а затем просто возвращает только V (значение).  

In [ ]:
def get(aMap, key, default=None):
    i, k, v = get_slot(aMap, key, default=default)
    return v  

# set
Чтобы установить пару ключ / значение все, что нужно сделать, это получить бакет, и добавить новую пару(ключ, значение) к нему, чтобы он мог быть найден позже.Тем не менее, я хочу, чтобы мой HashMap, позволял иметь только один ключ одновременно. Чтобы сделать это, сначала я должна найти бакет, затем проверить, может этот ключ уже существует.Если существует, то я заменяю его, а если нет, то добавляю новый.

In [3]:
def set(aMap, key, value):
    bucket = get_bucket(aMap, key)
    i, k, v = get_slot(aMap, key)

    if i >= 0:
        # ключ существует, заменить его
        bucket[i] = (key, value)
    else:
        # ключ не найден,добавьте,тем самым создав его
        bucket.append((key, value))

# delete
Чтобы удалить ключ, надо сначала использовать функцию get_bucket и далее искать ключ в нем,затем удалить его из списка. 

In [ ]:
def delete(aMap, key):
    bucket = get_bucket(aMap, key)

    for i in xrange(len(bucket)):
        k, v = bucket[i]
        if key == k:
            del bucket[i]
            break


#  list
Последняя функция которая выводит содержимое hashmap. Она получает каждый бакет, а затем проходит через каждый слот в бакете.

In [ ]:
def list(aMap):
    for bucket in aMap:
        if bucket:
            for k, v in bucket:
                print k, v

In [2]:
class hash_map(object):
    def __init__(self,num_buckets=256):
        
        aMap = []
        for i in range(0, num_buckets):
            aMap.append([])
        self.aMap = aMap
        
    def hash_key(self, key):
        return hash(key) % len(self.aMap)
    
    def get_bucket(self, key):
        bucket_id = self.hash_key( key)
        return self.aMap[bucket_id] 
   
    
    def get_slot(self, key, default=None):
        bucket = self.get_bucket( key)

        for i, kv in enumerate(bucket):
            k, v = kv
            if key == k:
                return i, k, v

        return -1, key, default
    

    def get(self, key, default=None):
        i, k, v = self.get_slot( key, default=default)
        return v
    
    def set(self, key, value):
        bucket = self.get_bucket( key)
        i, k, v = self.get_slot( key)

        if i >= 0:
            # ключ существует, заменить его
            bucket[i] = (key, value)
        else:
            # ключ не найден,добавьте,тем самым создав его
            bucket.append((key, value))
    
    def delete(self, key):
        bucket = self.get_bucket( key)

        for i in range(len(bucket)):
            k, v = bucket[i]
            if key == k:
                del bucket[i]
                break


    def __str__(self):
        res = "[ \n "
        for bucket in self.aMap:
            res += " [ "
            for kv in bucket:
                res += " " + str(kv) + " "
            res += " ], \n "
        res += " ] \n "
            
        return res

    def __repr__(self):
        return self.__str__()

#### Протестируем класс

In [3]:
hm = hash_map(8)

In [5]:
hm.set("tom","fish")
hm.set("jerry","cheese")
hm.set("bob","bone")
hm.set("chip","nuts")

In [6]:
print(hm)

[ 
  [  ('bob', 'bone')  ], 
  [  ], 
  [  ], 
  [  ], 
  [  ('chip', 'nuts')  ], 
  [  ], 
  [  ('tom', 'fish')  ('jerry', 'cheese')  ], 
  [  ], 
  ] 
 


In [166]:
hm.get("tom")

'fish'

In [162]:
hm.delete("tom")

In [156]:
print(hm)

[ 
  [  ], 
  [  ], 
  [  ('jerry', 'cheese')  ], 
  [  ('bob', 'bone')  ], 
  [  ('dale', 'nuts')  ], 
  [  ], 
  [  ], 
  [  ('olya', 'ochky')  ], 
  ] 
 


#### протестируем производительность

In [182]:
hm = hash_map()

для заполнения воспользуемся библиотекой для рандомной генерации данных https://github.com/joke2k/faker

In [188]:
from faker import Faker
fake = Faker()

In [189]:
names = []
for i in range(100):
    key = fake.name()
    names.append(key)
    val = (float(fake.latitude()),float(fake.longitude()))
    %%time hm.set(key,val)

Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 1 ms
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 500 µs
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 1 ms
Wall time: 1 ms
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 500 µs
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 501 µs
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 501 µs
Wall time: 0 ns
Wall time: 500 µs
Wall time: 509 µs
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 5

In [190]:
for i in names:
     %%time hm.get(i)

Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 500 µs
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 500 µs
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall time: 0 ns
Wall

По результатам небольшого теста видно насколько быстро работает наш словарь.


Вычислительная сложность основных операций :

func   | best    | worst
---    | ---     | ---
get    |	O(1) |	O(n)
set    |	O(1) |	O(n)
delete    |	O(1) |	O(n)